In [1]:
import pandas as pd

data = pd.read_parquet("data/gallica_presse_1_1.parquet")

In [2]:
data.head()

,index_id,file_id,ocr,title,date,author,page_count,word_count,character_count,text,corrected_text
0,940,bpt6k6441992x,99,Journal officiel de la République française. D...,1902-01-20,None,112,58622,356947,\nCHAMBRE DES DÉPUTÉS 71 législature. Session ...,CHAMBRE DES DÉPUTÉS 71e législature. Session o...
1,941,bpt6k2353312n,86,Journal de la Manche et de la Basse-Normandie ...,1913-03-01,None,22,31657,188831,\nDixième Année.— N©960 \nDIX CENTIMES LE NUME...,Dixième Année.— 1960 \nDIX CENTIMES LE NUMÉRO ...
2,942,bpt6k605149p,99,Le Petit Parisien : journal quotidien du soir,1922-09-22,None,22,27888,175974,\nLES DÉLIBÉRATIONS \nSORUlUESDID'lEir reprenn...,LES DÉLIBÉRATIONS \nSOURCUES DID'ELIRE reprenn...
3,943,bpt6k54904729,93,Bulletin du Photo-club de Constantine...,1894-12,Photo club de Constantine,34,2715,17059,\nDeuxième année \nDÉCEMBRE 1894 \nN° 16 \nFou...,Deuxième année\nDÉCEMBRE 1894\nN° 16\nPour les...
4,944,bpt6k611668p,99,L'Ouest-Éclair,1926-07-29,None,40,51163,317488,\nL'UNION SACREE POUR LE TRAVAIL \nNotre colla...,L'UNION SACRÉE POUR LE TRAVAIL \nNotre collabo...


In [ ]:
def write_datafile(filename, toks, model_desc="gpt-2"):
    """
    Saves token data as a .bin file, for reading in C.
    - First comes a header with 256 int32s
    - The tokens follow, each as uint16 (gpt-2) or uint32 (llama)
    """
    assert len(toks) < 2**31, "token count too large" # ~2.1B tokens
    assert model_desc in ["gpt-2", "llama-3"], f"unknown model descriptor {model_desc}"
    info = HEADERS_INFO[model_desc]
    # construct the header
    header = np.zeros(256, dtype=np.int32) # header is always 256 int32 values
    header[0] = info["magic"]
    header[1] = info["version"]
    header[2] = len(toks) # number of tokens after the 256*4 bytes of header
    # construct the data (numpy array of tokens)
    toks_np = np.array(toks, dtype=info["token_dtype"])
    # write to file
    num_bytes = (256 * 4) + (len(toks) * toks_np.itemsize)
    print(f"writing {len(toks):,} tokens to {filename} ({num_bytes:,} bytes) in the {model_desc} format")
    with open(filename, "wb") as f:
        f.write(header.tobytes())
        f.write(toks_np.tobytes())